<a href="https://colab.research.google.com/github/Avvonna/DL_sound_project/blob/main/demo_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 1\. Setup Repository and Libraries (~ 4 mins)

import os
import subprocess
import sys

REPO_NAME = "DL_sound_project"
GIT_URL = f"https://github.com/Avvonna/{REPO_NAME}.git"

def run_command(command, message):
    """
    Вспомогательная функция для тихой установки пакетов с помощью subprocess.
    """
    print(f"{message}...")
    try:
        subprocess.run(command, check=True, shell=True, capture_output=True, text=True)
        print(f"{message} - DONE")
    except subprocess.CalledProcessError as e:
        print(f"{message} - FAILED!")
        print("ERROR LOG:\n", e.stderr)
        raise e

# Repo
current_dir = os.path.basename(os.getcwd())
if current_dir == REPO_NAME:
    print(f"Already inside {REPO_NAME}. Skipping clone.")
    run_command("git pull", "Pulling latest changes")
else:
    if not os.path.exists(REPO_NAME):
        run_command(f"git clone {GIT_URL}", f"Cloning {REPO_NAME}")
    os.chdir(REPO_NAME)

# Dependencies
if not os.path.exists(".setup_complete"):
    if os.path.exists("requirements.txt"):
        with open("requirements.txt", "r") as f:
            reqs = f.readlines()

        new_reqs = []
        for req in reqs:
            # Пропускаем torch, numpy (в colab они более старые)
            if any(lib in req.lower() for lib in ["torch", "numpy"]):
                continue
            new_reqs.append(req)

        with open("requirements_colab.txt", "w") as f:
            f.writelines(new_reqs)

        # Ставим остальные библиотеки
        run_command("pip install -q -r requirements_colab.txt", "Installing project dependencies")

        # Синхронизируем версии torch и torch audio
        run_command("pip install --upgrade torchaudio", "Syncing Torch & TorchAudio")

        with open(".setup_complete", "w") as f:
            f.write("done")
        print("Dependencies installed.")
    else:
        print("requirements.txt not found!")
else:
    print("Dependencies already installed.")

print(f"\nCurrent Directory: {os.getcwd()}")
print("Setup complete!")

In [ ]:
# @title 2\. Download Pre-trained Weights

import gdown

FILE_ID = "12k5HUXaGLweeOGXSZqiqrhUG9CMjKujr"
OUTPUT_DIR = "saved"
MODEL_PATH = os.path.join(OUTPUT_DIR, "best_model.pth")

os.makedirs(OUTPUT_DIR, exist_ok=True)

if not os.path.exists(MODEL_PATH):
    print(f"Downloading weights to {MODEL_PATH}...")
    url = f'https://drive.google.com/uc?id={FILE_ID}'
    gdown.download(url, MODEL_PATH, quiet=True)
    print("Weights successfully downloaded")
else:
    print(f"Weights found at {MODEL_PATH}")

In [ ]:
# @title 3\. Helper Functions (Visualization & Inference)

import shutil
import soundfile as sf
import IPython.display as ipd
from pathlib import Path

def play_audio(file_path):
    """
    Читает и воспроизводит аудиофайл в Colab.
    """
    try:
        audio_array, sample_rate = sf.read(file_path)
        if audio_array.ndim > 1:
            audio_array = audio_array.T
        print(f"Audio: {os.path.basename(file_path)} | SR: {sample_rate}")
        ipd.display(ipd.Audio(audio_array, rate=sample_rate))
    except Exception as e:
        print(f"Could not play {file_path}: {e}")

def show_inference_results(audio_dir, prediction_dir, limit=None):
    """
    Проходит по файлам в audio_dir, ищет соответствующие .txt в prediction_dir
    и отображает результат (Аудио + Текст).

    Args:
        audio_dir (str): Путь к папке с аудио.
        prediction_dir (str): Путь к папке с предсказаниями (.txt).
        limit (int): Максимальное кол-во примеров для показа (полезно для больших датасетов).
    """
    files = sorted(os.listdir(audio_dir))

    # Фильтруем только аудио
    audio_extensions = ('.wav', '.mp3', '.flac')
    files = [f for f in files if f.lower().endswith(audio_extensions)]

    if not files:
        print("No audio files found to display.")
        return

    # Применяем лимит, если задан
    count = len(files)
    if limit and count > limit:
        print(f"Found {count} files. Showing first {limit} examples:\n")
        files = files[:limit]
    else:
        print(f"Showing results for {count} files:\n")

    for filename in files:
        file_id = Path(filename).stem
        audio_path = os.path.join(audio_dir, filename)
        pred_path = os.path.join(prediction_dir, f"{file_id}.txt")

        print("-" * 40)

        # 1. Играем аудио
        play_audio(audio_path)

        # 2. Печатаем текст
        if os.path.exists(pred_path):
            with open(pred_path, "r") as f:
                text = f.read().strip()
            print(f"\nText:\n{text.upper()}")
        else:
            print(f"\nPrediction file not found for ID: {file_id}")

def run_prediction(audio_path, model_path="saved/best_model.pth"):
    """
    Запускает inference.py через командную строку
    """

    base_dir = os.path.abspath(".")
    temp_dir = os.path.join(base_dir, "temp_inference_data")
    audio_sub = os.path.join(temp_dir, "audio")
    save_folder_name = "temp_predictions"

    # Очистка и создание папок
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    os.makedirs(audio_sub, exist_ok=True)

    # Очистка папки вывода
    full_save_path = os.path.join(base_dir, "data", "saved", save_folder_name)
    if os.path.exists(full_save_path):
        shutil.rmtree(full_save_path)

    # Копирование аудио
    path_obj = Path(audio_path)
    audio_name = path_obj.name
    shutil.copy(audio_path, os.path.join(audio_sub, audio_name))

    # Запуск команды
    command = [
        "python", "inference.py",
        f"inferencer.save_path={save_folder_name}",
        f"inferencer.from_pretrained={model_path}",
        f"inferencer.data_dir={temp_dir}",
        "inferencer.device=auto",
        "+writer.run_name=demo"
    ]

    print(f"Running inference on {audio_name}...")
    result = subprocess.run(command, capture_output=True, text=True)

    if result.returncode != 0:
        print("Inference Error:")
        print(result.stderr)
        return "ERROR"

    file_id = path_obj.stem
    expected_file = os.path.join(full_save_path, "inference", f"{file_id}.txt")

    if os.path.exists(expected_file):
        with open(expected_file, "r") as f:
            return f.read().strip()
    else:
        print(f"File not found: {expected_file}")
        return "ERROR: File not found"

print("Inference functions ready.")

Код ниже позволяет загружать собственные аудио-файлы и прогонять на них инференс.

Аудиофайлы для инференса должны быть в форматах `flac`, `mp3` или `wav`

Файлы сохряняются во временную директорию, далее используется структура `CustomDirDataset`

In [ ]:
# @title 4A. Run on uploaded audiofiles

import ipywidgets as widgets
from google.colab import files, output
import shutil
import subprocess
from pathlib import Path

# Загрузка файлов
print("Upload audio files (flac/wav/mp3):")
uploaded = files.upload()
output.clear()

if not uploaded:
    print("No files uploaded.")
else:
    print(f"Uploaded {len(uploaded)} files")

    # Настройка путей
    base_dir = os.path.abspath(".")
    input_root = os.path.join(base_dir, "temp_data")
    audio_dir = os.path.join(input_root, "audio")

    # Папка для сохранения результатов
    save_folder_name = "temp_data_predictions"
    full_save_path = os.path.join(base_dir, "data", "saved", save_folder_name)
    predictions_txt_dir = os.path.join(full_save_path, "inference")

    # Очистка и создание директорий
    if os.path.exists(input_root):
        shutil.rmtree(input_root)
    os.makedirs(audio_dir, exist_ok=True)

    if os.path.exists(full_save_path):
        shutil.rmtree(full_save_path)

    # Перемещение файлов в структуру датасета
    uploaded_filenames = list(uploaded.keys())
    for filename in uploaded_filenames:
        src_path = os.path.join(base_dir, filename)
        dst_path = os.path.join(audio_dir, filename)

        if os.path.exists(src_path):
            shutil.move(src_path, dst_path)

    # Inference
    print(f"Running inference...")

    command = [
        "python", "inference.py",
        f"inferencer.save_path={save_folder_name}",
        f"inferencer.from_pretrained=saved/best_model.pth",
        f"inferencer.data_dir={input_root}"
    ]

    try:
        result = subprocess.run(command, capture_output=True, text=True)

        if result.returncode != 0:
            print("Inference Error:")
            print(result.stderr)
        else:
            print("Inference finished.\n")

            show_inference_results(audio_dir, predictions_txt_dir, limit=None)

    except Exception as e:
        print(f"Critical Error during execution: {e}")

    finally:
        # Удаление исходных аудиофайлов из корня проекта
        # Они уже есть во временной директории
        if os.path.exists(filename):
            os.remove(filename)

Код ниже позволяет скачивать архив или папку с GDrive и прогонять инференс на ней.

Структура папки (или подпапки) должна иметь следующий формат:

```python
NameOfTheDirectoryWithUtterances
├── audio
│   ├── UtteranceID1.wav    # may be flac or mp3
│   ├── UtteranceID2.wav
│   .
│   .
│   .
│   └── UtteranceIDn.wav
└── transcriptions          # ground truth, may not exist
    ├── UtteranceID1.txt
    ├── UtteranceID2.txt
    .
    .
    .
    └── UtteranceIDn.txt
```

Аудиофайлы для инференса должны быть в форматах `flac`, `mp3` или `wav`

Вытащил первые попавшиеся файлы из `librispeech\dev-clean\84\121123` и перегнал некоторые из формата `flac` в `wav`, `mp3` и `m4a` (для проверки):

Вот ссылки на папки для быстрого доступа (могут быть в бане*):
- https://drive.google.com/drive/folders/1K-fpqyYsVxvleKqhpCQPfzV1_xC3RyEk?usp=drive_link (с транскрипциями - для расчета метрик)
- https://drive.google.com/drive/folders/1B2ZbI0jbzfjUOlqqWceib4R8pajMqE4u?usp=drive_link (без транскрипций - для их получения)

Вот ссылки на архивы с аналогичным содержанием:
- https://drive.google.com/file/d/1B5JUcSsPk1l_xe9c99iClJIUSzkXJh8h/view?usp=sharing (с транскрипциями)
- https://drive.google.com/file/d/1_9-mLQCYn9MCR3wm1RNxCA0bvt3rP6Ok/view?usp=sharing (без них)

**\(\*\) Частые скачивания с GDrive приводят к фризу диска**

In [ ]:
# @title 4B. Run on GDrive Dataset

import zipfile
import gdown

# @markdown Вставьте ссылку на Google Drive (на .zip файл или на папку):
gdrive_url = ""  # @param {type:"string"}

# Настройка путей
base_dir = os.path.abspath(".")
dataset_root = os.path.join(base_dir, "temp_custom_dataset")
save_folder_name = "custom_drive_predictions"
full_save_path = os.path.join(base_dir, "data", "saved", save_folder_name)

# Очистка предыдущих запусков
if os.path.exists(dataset_root):
    shutil.rmtree(dataset_root)
if os.path.exists(full_save_path):
    shutil.rmtree(full_save_path)

os.makedirs(dataset_root, exist_ok=True)

if not gdrive_url:
    print("Please paste a Google Drive link above. (zip or folder)")
else:
    try:
        # Скачивание (Folder или zip)
        if "drive.google.com" in gdrive_url and "/folders/" in gdrive_url:
            print("Detected Google Drive FOLDER.")
            gdown.download_folder(url=gdrive_url, output=dataset_root, quiet=True)
        elif "drive.google.com" in gdrive_url:
            print("Detected Google Drive FILE (assuming .zip).")
            zip_path = os.path.join(dataset_root, "dataset.zip")
            gdown.download(url=gdrive_url, output=zip_path, quiet=True, fuzzy=True)
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(dataset_root)
            os.remove(zip_path)
        else:
            # прямая ссылка
            print("Direct link detected.")
            zip_path = os.path.join(dataset_root, "dataset.zip")
            subprocess.run(
                ["wget", gdrive_url, "-q", "-O", zip_path],
                check=True
            )
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(dataset_root)
            os.remove(zip_path)

        # Поиск данных
        data_dir = None
        for root, dirs, files in os.walk(dataset_root):
            if "audio" in dirs:
                data_dir = root
                break

        if not data_dir:
            raise FileNotFoundError("Could not find 'audio' folder.")

        print(f"Dataset found at: {data_dir}")

        # Inference
        print("\nRunning Inference...")
        cmd_inf = [
            "python", "inference.py",
            f"inferencer.save_path={save_folder_name}",
            f"inferencer.from_pretrained=saved/best_model.pth",
            f"inferencer.data_dir={data_dir}"
        ]

        result = subprocess.run(cmd_inf, capture_output=True, text=True)

        if result.returncode != 0:
            print("Inference Error:")
            print(result.stderr)
        else:
            print("Inference finished.")

            # Paths
            gt_dir = os.path.join(data_dir, "transcriptions")
            preds_txt_dir = os.path.join(full_save_path, "inference")
            audio_source_dir = os.path.join(data_dir, "audio")

            # Metrics vs Visualization
            if os.path.exists(gt_dir) and os.path.isdir(gt_dir):
                print("\nGround truth found. Calculating WER/CER...")
                if os.path.exists(preds_txt_dir):
                    cmd_metrics = [
                        "python", "calc_metrics.py",
                        "--pred_dir", preds_txt_dir,
                        "--gt_dir", gt_dir
                    ]
                    metrics_res = subprocess.run(cmd_metrics, capture_output=True, text=True)
                    print(metrics_res.stdout)
                    if metrics_res.stderr: print("Metrics Errors:", metrics_res.stderr)
                else:
                    print("Error: Predictions not found.")

            if os.path.exists(preds_txt_dir):
                    show_inference_results(audio_source_dir, preds_txt_dir, limit=5)
            else:
                print("Predictions directory was not created.")

    except Exception as e:
        print(f"\nFAILED: {e}")


Код ниже позволяет записать аудио прямо внутри Colab и прогнать на нем инференс (надо говорить на английском)

*\* JS код писала нейросеть, но мне просто хотелось это реализовать, мне кажется прикольно :\)*

In [18]:
# @title 4C. Record Audio

from IPython.display import HTML, Audio, display
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np


# JS
AUDIO_HTML = """
<script>
var recordAudio = () => new Promise(async resolve => {
  // Создаем UI
  var div = document.createElement('div');
  var btn = document.createElement('button');

  btn.innerText = '🔴 Start Recording';
  btn.style.background = 'red';
  btn.style.color = 'white';
  btn.style.padding = '12px';
  btn.style.border = 'none';
  btn.style.borderRadius = '5px';
  btn.style.cursor = 'pointer';
  btn.style.fontSize = '16px';

  div.appendChild(btn);
  document.body.appendChild(div);

  // Инициализация
  const stream = await navigator.mediaDevices.getUserMedia({audio: true});
  const recorder = new MediaRecorder(stream);
  const chunks = [];

  // Собираем данные
  recorder.ondataavailable = e => chunks.push(e.data);

  // Когда запись остановлена
  recorder.onstop = () => {
    const blob = new Blob(chunks); // Браузер пишет в webm/ogg
    const reader = new FileReader();
    reader.readAsDataURL(blob);
    reader.onloadend = () => {
        div.remove(); // Удаляем кнопку
        resolve(reader.result); // Возвращаем данные в Python
    }
  };

  // Логика кнопки
  btn.onclick = () => {
    if (recorder.state === "inactive") {
      recorder.start();
      btn.innerText = "⬛ Stop Recording";
      btn.style.background = "black";
    } else {
      recorder.stop();
      stream.getTracks().forEach(track => track.stop()); // Выключаем микрофон
      btn.innerText = "Processing...";
      btn.style.background = "gray";
      btn.disabled = true;
    }
  }
});
</script>
"""

def record_audio_robust(filename='recording.wav'):
    # Внедряем JS функцию
    display(HTML(AUDIO_HTML))
    # Вызываем функцию и ждем Promise
    data = eval_js("recordAudio({})")

    # Декодируем
    binary = b64decode(data.split(',')[1])

    # Сохраняем временный файл (обычно это webm)
    temp_file = filename.replace('.wav', '.webm')
    with open(temp_file, 'wb') as f:
        f.write(binary)

    # Конвертация в WAV (16kHz, mono) через ffmpeg
    # -y: перезаписать, -vn: без видео, -ac 1: моно
    subprocess.run(['ffmpeg', '-y', '-i', temp_file, '-ar', '16000', '-ac', '1', filename],
                   stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    if os.path.exists(temp_file):
        os.remove(temp_file)

    return filename

base_dir = os.path.abspath(".")
input_root = os.path.join(base_dir, "temp_mic_data")
audio_dir = os.path.join(input_root, "audio")
save_folder_name = "mic_predictions"
full_save_path = os.path.join(base_dir, "data", "saved", save_folder_name)
filename = "my_voice.wav"

if os.path.exists(input_root):
    shutil.rmtree(input_root)
if os.path.exists(full_save_path):
    shutil.rmtree(full_save_path)

os.makedirs(audio_dir, exist_ok=True)

# Запись и Инференс
try:
    print("Click the button below to record (Click again to stop)")
    file_path = os.path.join(audio_dir, filename)

    record_audio_robust(file_path)

    if os.path.exists(file_path):
        print(f"Audio saved: {file_path}")

        print("\nRunning inference...")

        command = [
            "python", "inference.py",
            f"inferencer.save_path={save_folder_name}",
            f"inferencer.from_pretrained=saved/best_model.pth",
            f"inferencer.data_dir={input_root}",
            "inferencer.device=auto"
        ]

        result = subprocess.run(command, capture_output=True, text=True)

        if result.returncode != 0:
            print("Inference Error:")
            print(result.stderr)
        else:
            print("Inference finished.\n")

            predictions_txt_dir = os.path.join(full_save_path, "inference")
            if os.path.exists(predictions_txt_dir):
                show_inference_results(audio_dir, predictions_txt_dir)
            else:
                print("Predictions directory not found.")
    else:
        print("Error: File was not saved.")

except Exception as e:
    print(f"Error: {e}")

Click the button below to record (Click again to stop)


Audio saved: /content/DL_sound_project/temp_mic_data/audio/my_voice.wav

Running inference...
Inference finished.

Showing results for 1 files:

----------------------------------------
Audio: my_voice.wav | SR: 16000



Text:
ARM I WAS EEME ITSTUNNED WY THE WAY HE WALKED
